# Building a Classifier App with weblink
> Creating an image classifier app that is on the net! Using Jupyter notebooks, FastAi, Binder and Voila

- toc:true- branch: master
- badges: true
- comments: true
- author: Thomas H. Simm
- categories: [fastai, jupyter, Binder, Voila, myBinder, ThomasHSimm]

1) The Model Part

2) The Python Part Of the App

3) The Binder Part

## 1) The model Part

The model was run on https://colab.research.google.com with a GPU

Necessary to have a GPU for time

Some imports and installs

!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

from fastbook import *
from fastai.vision.widgets import *

Gets Azure search key to use Bing search API


key = os.environ.get('AZURE_SEARCH_KEY', 'keygoeshere')

I'm going to do a classifier for holiday types

destas={'beach':{'beach','tropical','sea','beach holidays'},
        'snow':{'ski','snowboard','snow','ski holidays'},
        'countryside':{'lakes mountains','countryside','forest','fields'},
        'city':{'city','cities','bars','buildings'},
        'safari':{'safari','safari holidays','safari park','safari africa'}}

Creates a folder containing images for each type

Need to add to dir with different searches not delete and add new stuff each time

For eaach holiday type- go through the search topics and add the results together before downloading images to the holiday type folder

This gives 600 pics per holiday type

path = Path('Destinations')
try:
    path.mkdir()
except:
    pass